In [1]:
import numpy as np
import tensorflow as tf
from tensorboard.plugins.hparams import api as hp

### Importing the data

In [3]:
data_train = np.load(r"C:\Users\Shubham\Desktop\Projects\365 Data Science\Machine learning\CNN\Full Dataset\Primary categories - Train.npz")
data_test = np.load(r"C:\Users\Shubham\Desktop\Projects\365 Data Science\Machine learning\CNN\Full Dataset\Primary categories - Test.npz")
data_val = np.load(r"C:\Users\Shubham\Desktop\Projects\365 Data Science\Machine learning\CNN\Full Dataset\Primary categories - Validation.npz")

In [4]:
images_train = data_train["images"]
labels_train = data_train["labels"]

images_val = data_val["images"]
labels_val = data_val["labels"]

images_test = data_test["images"]
labels_test = data_test["labels"]

### Scaling the images

In [6]:
# scaling the pixels between 0 and 1
images_train = images_train/255.0
images_val = images_val/255.0
images_test = images_test/255.0

### Model outline and Hyperparameter tuning

In [8]:
BATCH_SIZE = 64
EPOCHS = 15

In [9]:
HP_FILTER_SIZE = hp.HParam("filter_size", hp.Discrete([3,5,7]))
HP_FILTER_NUM = hp.HParam("filter_num", hp.Discrete([64,96,128]))

METRIC_ACCURACY = "accuracy"

with tf.summary.create_file_writer(r"logs/Model 1/hparam_tuning").as_default():
    hp.hparams_config(
        hparams = [HP_FILTER_SIZE, HP_FILTER_NUM],
        metrics = [hp.Metric(METRIC_ACCURACY, display_name="accuracy")]
    )
    

In [10]:
def train_test_model(hparams,session_num):

    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(hparams[HP_FILTER_NUM], hparams[HP_FILTER_NUM], activation="relu", input_shape = (120,90,3)),
        tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
        tf.keras.layers.Conv2D(hparams[HP_FILTER_NUM], hparams[HP_FILTER_SIZE], activation="relu"),
        tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(3)
    ])

    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

    model.compile(optimizer = "adam", loss = loss_fn, metrics = ["accuracy"])
    log_dir = "logs\\Model 1\\fit\\" + "run{}".format(session_num)

    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir = log_dir, histogram_freq = 1, profile_batch= True)
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_loss",
                                                     mode= "auto", min_delta = 0, patience =2, verbose = 0, restore_best_weights = True)

    model.fit(images_train, labels_train,
             epochs = EPOCHS, batch_size = BATCH_SIZE, callbacks = [tensorboard_callback, early_stopping], 
              validation_data = (images_val, labels_val), verbose = 2)
    _, accuracy = model.evaluate(images_val, labels_val)

    #model.save(r"saved_models/Model 1/run-{}".format(session_num))
    return accuracy
    

In [14]:
### Run function to log the hparams summary and accuracy
def run(log_dir, hparams, session_num):
    with tf.summary.create_file_writer(log_dir).as_default():
        hp.hparams(hparams) # record the values used in the trial
        accuracy = train_test_model(hparams, session_num)
        tf.summary.scalar(METRIC_ACCURACY, accuracy, step =1)
    
    

In [ ]:
session_num = 0 

for filter_size in HP_FILTER_SIZE.domain.values:
    for filter_num in HP_FILTER_NUM.domain.values:

        hparams = {HP_FILTER_SIZE : filter_size,
                  HP_FILTER_NUM : filter_num}
        run_name = "run-%d" % session_num
        print("--- Starting trial :%s" % run_name)
        print({h.name : hparams[h] for h in hparams})
        run("logs/Model 1/hparam_tuning/" + run_name, hparams, session_num)
        session_num += 1

    

### Visualizing the tensorboard

In [ ]:
%load_ext tensorboard
%tensorboard --log_dir "logs/Model 1/hparams_tuning"

In [ ]:
%load_ext tensorboard
%tensorboard --log_dir "logs/Model 1/fit"